In [27]:
from common_utils.utils.config import Config
from common_utils.io.data_access.data_access_factory import DataAccessFactory
# from axpo_trading.forecast.forecast_preprocess_iberia import preproces_ufis
from common_utils.utils import utils, utils_io, utils_date
from axpo_trading.forecast import forecast_sql_preprocess_iberia
from axpo_trading.forecast import forecast_preprocess_iberia
import pandas as pd
import warnings
warnings.filterwarnings('ignore')
import os
wind_path = "/home/jovyan/projects/AdvancedAnalytics-UseCase-Wind"
os.chdir(wind_path)

os.environ['CONFIG_DIR'] = 'config_files'
os.environ['AUTH_CONFIG_DIR'] = 'auth'
os.environ['AZURE_STORAGE_ACCOUNT_RAW_CONTAINER_NAME_WIND_RAW'] = 'raw'
os.environ['AZURE_STORAGE_ACCOUNT_RAW_CONTAINER_NAME_WIND_STAGING'] = 'staging'
os.environ["AZURE_SQL_SHARED_RAW_SERVER"] = 'axso-prod-appl-aa-prod-shared-sql-secondary.database.windows.net'
# os.environ["AZURE_SQL_SHARED_RAW_SERVER"] = 'axso-prod-appl-aa-prod-shared-sql.database.windows.net'
os.environ["AZURE_SQL_SHARED_RAW_DATABASE"] = 'axso-prod-appl-aa-prod-shared-raw-sqldb'
os.environ["N_THREADS_SQL"] = "1"

# DEV
os.environ['ENV'] = 'azure_iberia_k8s_dev'
# BLOB DEV
os.environ['AZURE_STORAGE_ACCOUNT_DATA_NAME'] = 'axsonpaadevdslabdls'
os.environ['AZURE_STORAGE_ACCOUNT_RAW_NAME'] = 'axsoprodaaprodshareddls-secondary'
os.environ['AZURE_STORAGE_ACCOUNT_DATA_CONTAINER_NAME_WIND_REFINED'] = 'wind-refined'
os.environ['AZURE_STORAGE_ACCOUNT_DATA_CONTAINER_NAME_WIND_RESULTS'] = 'wind-results'


1. Get Eolic UFIS for the given dates

In [28]:
# date_from = "2020-08-01"
date_from = "2019-01-01"
date_to = "2022-10-30"

In [29]:
eolic_ufis_query = "SELECT CDUNIFIS, CDUNIPROG, NUMREGISTRO, FEALTA, FEBAJA, provincia, ubicacion, Latitud as lat, Longitud as lon, PTMAXIMA, PTMINIMA, TIPOUNIFIS, COMISIONMWH, codCliente, cliente, \
	 						COEFREPARTO, COEFPERDIDAS, INDCOSTEDES, DISTRIBUIDORA, FACTURAR_MI, CONSG_REACTIVA, FEULTMOD \
					FROM [HISTORIAN].[DatosInstalaciones] A \
					INNER JOIN (\
						SELECT * FROM [EGL].[SYS_UNIDAD_FISICA] WHERE TIPOUNIFIS='EO' \
					) B \
					ON A.codUFisica = B.CDUNIFIS"

config_dict = Config.get_config()
data_config = config_dict["data_access_factory"]
factory = DataAccessFactory()

source = factory.get(data_config["pre_master_forecast"]["source"])
eo_ufis = source.get(eolic_ufis_query)

# Can't use the existing one since it skips lat lon data :(
def preproces_ufis(
    date_to, ufi_up, date_from=None, add_limit=False, add_datetime_col=False, usecase=None, valid_units=None
):
    """
    Preprocess ufis table into standardized (std_ufi) table

    :param date_to: end date (type: str)
    :param ufi_up: raw ufi table (type: pd.DataFrame)
    :param date_from: start date (type: str)
    :param add_limit: flag to add extra capacity limitations. Expands the table to hourly resolution (type: boolean)
    :param add_datetime_col: flag to add the column datetime (type: boolean)
    :param usecase: usecase to use to add extra capacity limitations (type: str)
    :return: processed table (type: pd.DataFrame)
    """

    if valid_units is not None:
        ufi_up = ufi_up[ufi_up["CDUNIFIS"].isin(valid_units)]

    # Converting columns to datetime
    # logger.info("Casting columns to date format")
    ufi_up["FEALTA"] = pd.to_datetime(ufi_up["FEALTA"], format="%Y-%m-%d %H:%M:%S.%f")
    ufi_up["FEBAJA"] = pd.to_datetime(ufi_up["FEBAJA"], format="%Y-%m-%d %H:%M:%S.%f")

    # Filling NA to_dates, and cliping dates over today, to today
    date_to_plus_one = pd.to_datetime(date_to) + pd.Timedelta(1, unit="D")
    ufi_up = ufi_up[ufi_up["FEALTA"] <= date_to_plus_one].copy()
    ufi_up.loc[ufi_up["FEBAJA"].isnull(), "FEBAJA"] = date_to_plus_one
    ufi_up.loc[ufi_up["FEBAJA"] > date_to_plus_one, "FEBAJA"] = date_to_plus_one
    # logger.info("Expanding UFI table")

    # Expand dates
    cols_use = [
        "TIPOUNIFIS",
        "COMISIONMWH",
        "COEFREPARTO",
        "COEFPERDIDAS",
        "INDCOSTEDES",
        "DISTRIBUIDORA",
        "FACTURAR_MI",
        "CONSG_REACTIVA",
        "provincia",
        "ubicacion",
        "lat",
        "lon",
        "codCliente"
    ]

    # ufi_up_expanded = utils.expand_dates(ufi_up,
    ufi = utils_date.expand_dates(
        ufi_up,
        key_cols=["CDUNIFIS", "CDUNIPROG", "PTMAXIMA", "PTMINIMA"] + cols_use,
        from_col="FEALTA",
        to_col="FEBAJA",
        include_last=False,
    )

    # Consolidating into ufi-uo table renaming and filtering
    ufi.rename(
        columns={"CDUNIFIS": "ufi", "CDUNIPROG": "up", "PTMAXIMA": "p_max", "PTMINIMA": "p_min"},
        inplace=True,
    )
    ufi = ufi[["ufi", "up", "date", "p_max", "p_min"] + cols_use]
    ufi["date"] = pd.to_datetime(ufi["date"], format="%Y-%m-%d %H:%M:%S.%f")
    ufi.drop_duplicates(subset=["ufi", "date"], keep="last", inplace=True)
    if date_from is not None:
        ufi = ufi[ufi["date"] >= pd.to_datetime(date_from)]
    # # Add extra capacity limitations
    # if add_limit:
    #     ufi = add_ufi_limit(ufi)

    # Transform to QH
    # if add_datetime_col:
    #     ufi = utils_date.add_datetime_columns(ufi, hour_band_right=True)

    return ufi

ufi = preproces_ufis(date_to, eo_ufis, date_from, valid_units=eo_ufis["CDUNIFIS"].unique())

2023-01-25 19:13:07,451 - MainThread - [INFO] - b'[AD AUTH] - get_token at line 150: Token for resource axso-prod-appl-aa-prod-shared-raw-sqldb. Valid until 2023-01-26T18:13:06'


In [30]:
ufi.to_csv("data/tfm/ufi.csv", index=False)

2. Get the meterings for the given dates (target for the model)

In [31]:
# Meterings (target)
meterings = forecast_sql_preprocess_iberia.process_sql_meterings_stand_alone(
    date_from=date_from,
    date_to=date_to,
    real_time=False,
)

meterings = forecast_preprocess_iberia.preprocess_metering(date_from, date_to, meterings, ufi)

2023-01-25 19:13:16,578 - MainThread - [INFO] - b'[FORECAST SQL PREPROCESS IBERIA] - process_sql_meterings_stand_alone at line 429: date_from: 2019-01-01'
2023-01-25 19:13:16,597 - MainThread - [INFO] - b'[FORECAST SQL PREPROCESS IBERIA] - process_sql_meterings_stand_alone at line 430: date_to: 2022-10-30'
2023-01-25 19:13:17,054 - MainThread - [INFO] - b'[FORECAST SQL PREPROCESS IBERIA] - pull_metering_from_sql at line 529: Establishing connection to database'
2023-01-25 19:13:17,074 - MainThread - [INFO] - b'[FORECAST SQL PREPROCESS IBERIA] - pull_metering_from_sql at line 538: Downloading tables'
2023-01-25 19:15:02,452 - MainThread - [INFO] - b'[FORECAST PREPROCESS IBERIA] - preprocess_metering at line 898: Pre-processing meterings'


In [ ]:
meterings.to_csv("data/tfm/meterings.csv", index=False)

3. Get the telemetry for the given dates. It will help the model to correct forecast predictions

In [32]:
telemetry = forecast_sql_preprocess_iberia.pull_telemetry_from_sql(ufi["ufi"].unique(), date_from, date_to)
telemetry = forecast_preprocess_iberia.preprocess_telemetry(telemetry, ufi, portfolio_level=False, upsample=False)

2023-01-25 19:15:14,524 - MainThread - [INFO] - b'[FORECAST SQL PREPROCESS IBERIA] - pull_telemetry_from_sql at line 673: [SQL TELEMETRY] Establishing connection to database'
2023-01-25 19:15:15,051 - MainThread - [INFO] - b'[FORECAST SQL PREPROCESS IBERIA] - get_telemetry_for_tags at line 747: Downloading telemetry data from Historian database for        93 UFIs between 2019-01-01 and 2022-10-30...'
2023-01-25 19:15:15,076 - MainThread - [INFO] - b'[FORECAST SQL PREPROCESS IBERIA] - get_telemetry_for_tags at line 774: [SQL TELEMETRY] RUNNING QUERY'


In [33]:
telemetry["ufi"].unique()

array(['AMPLESC', 'ARTEIXO', 'ATALAYA', 'BAYO', 'AZUBIAS', 'BANDELE',
       'CHANTAD', 'AEROGEN', 'CPELAOS', 'PEARBO', 'BRULLES', 'ASNEVES',
       'PEOCHAO', 'COFRENT', 'PECOUTE', 'FEIXOS', 'PECUEVA', 'DEHESII',
       'DEHEII', 'OTERO', 'ELLLAN', 'ESCANDO', 'ESQUILE', 'FARELO',
       'CERCEDA', 'ELGALLO', 'GECAMA', 'HINOJAI', 'HINOJII', 'CALERA',
       'GRAIADE', 'PEIRIXO', 'CAMPANA', 'ESE', 'LACAYA', 'FRAILA',
       'HERRERI', 'PELALOM', 'LALOMBA', 'LAMESA', 'LARUYA', 'LASORDA',
       'TRAPERA', 'COMES', 'CERROS', 'MALPICA', 'MONDONE', 'POTRA',
       'ABELLA', 'MCABEZA', 'MONTERO', 'MONTCEO', 'CIERZO1', 'CIERZO2',
       'MONTOUT', 'MUDEFER', 'DEFERII', 'CADEIR', 'CARRACE', 'NEDAPE',
       'PEAPAS', 'SASDOI', 'SASDON', 'PEOUROL', 'SPADRON', 'PTEJADA',
       'PEDREGA', 'PEDREGB', 'PEDREGD', 'PGRANDE', 'PECORTI', 'PPOZA1',
       'PICADOR', 'PPOZA2', 'RODERA', 'ROMERA', 'SABUCED', 'SANJOSE',
       'PESLOA', 'PESLOB', 'PESLOC', 'PESLOD', 'SERRETA', 'TIGUEIR',
       'VILACHA',

In [8]:
telemetry.to_csv("data/tfm/telemetry.csv", index=False)

4. Get Forecasts to improve


In [34]:
data_config = Config.get_config()["data_access_factory"]
factory = DataAccessFactory()

sink = factory.get(data_config["master_overcost"]["source"])

path = "forecast/research/raw_forecasts_eolic"
# date_from_fcst = "2020-07-31"
# date_to_fcst = "2022-02-01"

markets_to_run = ["MD","MI1","MI2","MI3","MI4","MI5","MI6","MI7","mic_1","mic_2","mic_3","mic_4","mic_5","mic_6","mic_7","mic_8","mic_9","mic_10","mic_11","mic_12","mic_13","mic_14","mic_15","mic_16","mic_17","mic_18","mic_19","mic_20","mic_21","mic_22","mic_23","mic_24","tertiary_1","tertiary_2","tertiary_3","tertiary_4","tertiary_5","tertiary_6","tertiary_7","tertiary_8","tertiary_9","tertiary_10","tertiary_11","tertiary_12","tertiary_13","tertiary_14","tertiary_15","tertiary_16","tertiary_17","tertiary_18","tertiary_19","tertiary_20","tertiary_21","tertiary_22","tertiary_23","tertiary_24"]

# This takes too long so I did it on the cluster (run/iberia/fcst-on-cluster)
# forecasts = forecast_sql_preprocess_iberia.preprocess_sql_forecast(
#     date_from, date_to, markets_to_run, stand_alone=True, append_to_master=False, valid_ufis=ufi["ufi"].unique()
# )
# Instead we read the results from blob (generated in run/iberia/fcst-on-cluster)
forecasts = utils_io.load_monthly(
    path=path,
    date_col="date",
    date_from=date_from,
    date_to=date_to,
    data_access=sink,
)

forecasts = forecast_preprocess_iberia.preprocess_std_forecast(forecasts, False, ufi)

2023-01-25 19:17:38,910 - MainThread - [INFO] - b'[UTILS IO] - load_monthly at line 176: Loading monthly from : forecast/research/raw_forecasts_eolic'
2023-01-25 19:17:38,929 - MainThread - [INFO] - b'[UTILS IO] - format_dates at line 465: 2019-01-01'
2023-01-25 19:17:38,994 - MainThread - [INFO] - b"[UTILS IO] - filter_files_load_monthly at line 448: Files in path: ['raw_forecasts_eolic___202206_0.h5', 'raw_forecasts_eolic___202208_0.h5', 'raw_forecasts_eolic___201912_0.h5', 'raw_forecasts_eolic___201911_0.h5', 'raw_forecasts_eolic___201903_0.h5', 'raw_forecasts_eolic___201909_0.h5', 'raw_forecasts_eolic___202204_0.h5', 'raw_forecasts_eolic___202210_0.h5', 'raw_forecasts_eolic___202011_0.h5', 'raw_forecasts_eolic___202008_0.h5', 'raw_forecasts_eolic___202111_0.h5', 'raw_forecasts_eolic___202003_0.h5', 'raw_forecasts_eolic___202202_0.h5', 'raw_forecasts_eolic___201901_0.h5', 'raw_forecasts_eolic___202107_0.h5', 'raw_forecasts_eolic___202001_0.h5', 'raw_forecasts_eolic___201907_0.h5', '

In [13]:
forecasts.to_csv("data/tfm/forecasts.csv", index=False)

5. Create premaster table.

In [35]:
unit_col_name = "ufi"
datetime_cols =  ["datetime","date","hour","minute"]
col_zone= "zone"

# Dates
forecasts, meterings, ufi = forecast_preprocess_iberia.cast_dates_before_master(forecasts, meterings, ufi)
forecasts, meterings, ufi = forecast_preprocess_iberia.filter_dates(date_from, date_to, forecasts, meterings, ufi)

# Prepare data for master
forecast_preprocess_iberia.drop_duplicates(forecasts, meterings, ufi)
forecasts = forecast_preprocess_iberia.prepare_forecast_for_master(forecasts)

print("Appending metering to master")
master = forecasts.merge(meterings, on=datetime_cols + ["ufi"], how="left")
master["zone"] = "iberia"
print("Appending UFI information")
master = master.merge(ufi, on=["ufi", "date"], how="left")

# Prepare data for master
telemetry["date"] = pd.to_datetime(telemetry["date"]).dt.date
telemetry = telemetry[
    (telemetry["date"] >= pd.to_datetime(date_from).date() - pd.Timedelta(1, unit="d"))
    & (telemetry["date"] <= pd.to_datetime(date_to).date())
]
telemetry.drop_duplicates(subset=["datetime", "date", "hour", "ufi"], inplace=True)
telemetry = telemetry.reset_index(drop=True)
# Add telemetry data available at market time: Telemetry is inserted with 1h delay
telemetry["datetime_telem_available_delete"] = telemetry["datetime"] + pd.Timedelta("1h")

master = master.merge(
    telemetry,
    left_on=["datetime_market"] + [unit_col_name],
    right_on=["datetime_telem_available_delete"] + [unit_col_name],
    how="left",
    suffixes=("", "_delete"),
)


2023-01-25 19:25:06,444 - MainThread - [INFO] - b'[FORECAST PREPROCESS IBERIA] - drop_duplicates at line 1469: Making sure there are no duplicates in std tables'
Appending metering to master
Appending UFI information


In [36]:
master[["datetime_market","datetime","ufi","telemetry","forecast","metering"]]

,datetime_market,datetime,ufi,telemetry,forecast,metering
0,2019-01-01 11:00:00,2019-01-02 00:00:00,TRAPERA,NaN,0.0,0.000
1,2019-01-01 11:00:00,2019-01-02 01:00:00,TRAPERA,NaN,0.0,0.000
2,2019-01-01 11:00:00,2019-01-02 02:00:00,TRAPERA,NaN,0.0,0.000
3,2019-01-01 11:00:00,2019-01-02 03:00:00,TRAPERA,NaN,0.0,0.000
4,2019-01-01 11:00:00,2019-01-02 04:00:00,TRAPERA,NaN,0.1,0.000
...,...,...,...,...,...,...
26815780,2022-10-30 19:00:00,2022-10-30 20:00:00,PELALIN,NaN,0.4,0.871
26815781,2022-10-30 20:00:00,2022-10-30 21:00:00,PELALIN,NaN,0.8,1.261
26815782,2022-10-30 21:00:00,2022-10-30 22:00:00,PELALIN,NaN,1.3,1.866
26815783,2022-10-30 22:00:00,2022-10-30 23:00:00,PELALIN,NaN,1.7,2.252


In [23]:
master.to_csv("data/tfm/premaster_2.csv", index=False)

In [37]:
utils_io.save_monthly(
    table=master,
    path="forecast/research/premaster_eolic",
    date_col="datetime",
    table_name="premaster_eolic",
    file_format="hdf",
    data_access=sink,
)

2023-01-25 19:28:44,499 - MainThread - [INFO] - b'[UTILS IO] - save_monthly at line 87: Saving table in forecast/research/premaster_eolic'
2023-01-25 19:28:49,531 - MainThread - [INFO] - b'[MULTITHREADING] - apply_multithread at line 55: Using 1 threads'
2023-01-25 19:28:49,552 - MainThread - [INFO] - b'[UTILS IO] - save_month at line 124: Saving date 201901'
2023-01-25 19:28:58,757 - MainThread - [INFO] - b'[UTILS IO] - save_month at line 124: Saving date 201902'
2023-01-25 19:29:04,264 - MainThread - [INFO] - b'[UTILS IO] - save_month at line 124: Saving date 201903'
2023-01-25 19:29:10,272 - MainThread - [INFO] - b'[UTILS IO] - save_month at line 124: Saving date 201904'
2023-01-25 19:29:15,763 - MainThread - [INFO] - b'[UTILS IO] - save_month at line 124: Saving date 201905'
2023-01-25 19:29:21,436 - MainThread - [INFO] - b'[UTILS IO] - save_month at line 124: Saving date 201906'
2023-01-25 19:29:27,117 - MainThread - [INFO] - b'[UTILS IO] - save_month at line 124: Saving date 2019